In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pathlib
import os
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import Sequential, layers
from tensorflow.keras.preprocessing.image import load_img
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [10]:
flowers_dir = './Crop_img/'
flower_labels = sorted(os.listdir(flowers_dir))
flower_labels

['Babi',
 'Calimerio',
 'Chrysanthemum',
 'Hydrangeas',
 'Lisianthus',
 'PingPong',
 'Rosy',
 'Tana']

In [ ]:
X = []
y = []

img_height = 299
img_width = 299

fig,ax = plt.subplots(2, 4, figsize=(20,10))
ax=ax.flatten()

flowers_dir_path_object = pathlib.Path(flowers_dir)

for label_number,flower in enumerate(flower_labels):
    # get all paths of images inside flower subdirectory
    flower_images_paths = list(flowers_dir_path_object.glob(f'{flower}/*.jpg'))
    ax[label_number].set_title(flower.capitalize())
    ax[label_number].axis('off')
    flower_img = np.array(load_img(flower_images_paths[0].__str__()))
    ax[label_number].matshow(flower_img)
    # print(flower_images_paths[0].__str__())
    for image in flower_images_paths:
        # load and resize image
        resized_image = load_img(str(image),target_size=(img_height, img_width))
        # append features in X and labels in y
        X.append(np.array(resized_image))
        y.append(label_number)
        


In [ ]:

def scanData():
    data_count = pd.DataFrame(data={"flower": [], "count": []})
    meta_data_df = pd.DataFrame(data={"image_path": [], "flower_type": []})
    folder_path = flowers_dir

    subdirectories = next(os.walk(folder_path))[1]

    print(subdirectories)

    for i in subdirectories:
        sub = next(os.walk(folder_path+"/"+i))
        
        new_meta_data = pd.DataFrame(data={"image_path": [ (sub[0] + "/" + j) for j in sub[2]] , "flower_type": i})
        meta_data_df = pd.concat([meta_data_df, new_meta_data], ignore_index=True)
        
        new_data = pd.DataFrame(data={"flower": i, "count": len(sub[2])}, index=[0])      
        data_count = pd.concat([data_count, new_data], ignore_index=True)
        print(i, len(sub[2]), len([ sub[0] + j for j in sub[2]]))

    return data_count, meta_data_df

img_count, meta_data = scanData()
img_count

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(data=img_count, x='flower', y='count')

In [ ]:
# plt.imshow(new_image = tf.image.per_image_standardization(mpimg.imread(meta_data['image_path'][i])))

In [ ]:
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import load_img
import random
import pathlib

fig, ax = plt.subplots(2, 4, figsize=(20,20), sharex=False)
ax = ax.flatten()

flowers_dir = './Flowers_cleaned/'
flower_labels = sorted(os.listdir(flowers_dir))
flower_labels
flowers_dir_path_object = pathlib.Path(flowers_dir)
for label_number,flower in enumerate(flower_labels):
    
    # get all paths of images inside flower subdirectory
    flower_images_paths = list(flowers_dir_path_object.glob(f'{flower}/*.jpg'))
    
    ax[label_number].set_title(flower.capitalize())
    ax[label_number].axis('off')
    flower_img = np.array(load_img(flower_images_paths[0].__str__()))
    ax[label_number].matshow(flower_img)


In [ ]:
def split_dataset(data):
    TEST_RATIO = 0.2
    train_data, test_data = train_test_split(data, test_size=TEST_RATIO, random_state=100)
    # train_data, val_data = train_test_split(train_data, test_size=TEST_RATIO, random_state=100)

    print("Train data : {}, Test Data: {}".format(
        train_data.shape[0], test_data.shape[0]))
    return train_data, test_data

train_data, test_data = split_dataset(meta_data)

plt.figure(figsize=(20,10))
sns.histplot(data=test_data, x='flower_type')

# Data split

In [22]:
X = np.array(X)
y = np.array(y)

print('X shape: {}'.format(X.shape))
print('y shape: {}'.format(y.shape))

X shape: (10572, 299, 299, 3)
y shape: (10572,)


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

print('X train shape: {}'.format(X_train.shape))
print('X test shape: {}'.format(X_test.shape))

X train shape: (7929, 299, 299, 3)
X test shape: (2643, 299, 299, 3)


In [15]:
def plot_learning_curve(train_loss, val_loss, train_metric, val_metric, metric_name='Accuracy'):
    plt.figure(figsize=(10,5))
    
    plt.subplot(1,2,1)
    plt.plot(train_loss, 'r--')
    plt.plot(val_loss, 'b--')
    plt.xlabel("epochs")
    plt.ylabel("Loss")
    plt.legend(['train', 'val'], loc='upper left')

    plt.subplot(1,2,2)
    plt.plot(train_metric, 'r--')
    plt.plot(val_metric, 'b--')
    plt.xlabel("epochs")
    plt.ylabel(metric_name)
    plt.legend(['train', 'val'], loc='upper left')
    
    plt.axhline(y = max(train_metric), color = 'r', linestyle = '-')
    plt.axhline(y = max(val_metric), color = 'b', linestyle = '-')

    plt.show()

In [28]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Add, Flatten, Dense
from tensorflow.keras.models import Model

def identity_block(X, filters):
    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, filters, s=2):
    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid')(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def ResNet50(input_shape=(224, 224, 3), classes=1000):
    X_input = Input(input_shape)

    X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, filters=[64, 64, 256], s=1)
    X = identity_block(X, filters=[64, 64, 256])
    X = identity_block(X, filters=[64, 64, 256])

    X = convolutional_block(X, filters=[128, 128, 512], s=2)
    X = identity_block(X, filters=[128, 128, 512])
    X = identity_block(X, filters=[128, 128, 512])
    X = identity_block(X, filters=[128, 128, 512])

    X = convolutional_block(X, filters=[256, 256, 1024], s=2)
    X
    
    


In [29]:
model = ResNet50(input_shape=(img_height, img_width, 3), classes=8)

In [30]:
sgd_opt = tf.keras.optimizers.SGD(
    learning_rate=0.0001,
    momentum=0.8,
)
model.compile( optimizer=sgd_opt,loss='sparse_categorical_crossentropy',metrics='accuracy')
model.summary()

AttributeError: 'NoneType' object has no attribute 'compile'

In [16]:
# history = model.fit(train_generator, validation_data = test_generator,
#                               epochs=20, verbose=2, class_weight=class_weights_dict)


In [ ]:
plot_learning_curve(history.history['loss'], history.history['val_loss'], 
                    history.history['accuracy'], history.history['val_accuracy'], 
                    metric_name='Accuracy')

In [ ]:
# cm = sklearn.metrics.multilabel_confusion_matrix(test_generator.classes, model.predict(test_generator))
prediction = model.predict(test_generator)
index_max = prediction.argmax(axis=1)
print(index_max.shape)
# print(prediction[index_max])

In [ ]:
prediction

In [ ]:
prediction.shape

In [ ]:

# fix, ax = plt.subplots(2, 4, sharex=True, figsize=(20,10))
# ax = ax.flatten()
# for i in range(7):
    # sns.histplot(data=new, ax=ax[i])

In [ ]:
cm = tf.math.confusion_matrix(test_generator.classes, index_max, num_classes=8)
# cm = sklearn.metrics.multilabel_confusion_matrix(test_generator.classes, index_max)


print(cm)

In [ ]:
plt.figure(figsize=(20,15))
fx=sns.heatmap(cm, annot=True, fmt=".2f",cmap="GnBu")
fx.set_title('Confusion Matrix \n');
fx.set_xlabel('\n Predicted Values\n')
fx.set_ylabel('Actual Values\n');
fx.xaxis.set_ticklabels(train_generator.class_indices)
fx.yaxis.set_ticklabels(train_generator.class_indices)
plt.show()
